In [30]:
import os
import pandas as pd

df = pd.DataFrame(columns=['observation'])

path = 'C:\\Users\\Admin\\Desktop\\EnronMeetings-XML\\train'
dir_list = os.listdir('C:\\Users\\Admin\\Desktop\\EnronMeetings-XML\\train')
dir_list.sort()
print (dir_list)


['bunch1', 'bunch2', 'bunch3']


In [31]:
#getting content of email from all train text files
for i in dir_list:
    file_list = os.listdir('C:\\Users\\Admin\\Desktop\\EnronMeetings-XML\\train\\' + i)
    for f in file_list:
        path = 'C:\\Users\\Admin\\Desktop\\EnronMeetings-XML\\train\\' + i + '/' + f
        with open(path) as fi:
            line = ''
            found = False
            for word in fi.read().split():
                if word in ['7Bit', '7bit']:
                    found = True
                if found:
                    line = line + ' ' + word

            current_df = pd.DataFrame({'observation': [line]})
            df = df.append(current_df, ignore_index=True)

In [32]:
df

,observation
0,7bit Bcc: shona.wilson@enron.com This meeting...
1,7bit Review available services with OATI for ...
2,"7bit Concerning traders news information, and..."
3,"7bit As per our conversation, we will be meet..."
4,7bit Traders News Meeting <true_name> Chuck <...
...,...
724,"7bit Bcc: jay.webb@enron.com, legal <.taylor@..."
725,7bit Thought you may be interested in this as...
726,"7bit Bcc: debra.bailey@enron.com, rika.imai@e..."
727,"7bit Bcc: paul.racicot@enron.com, sheila.twee..."


In [35]:
from tqdm import tqdm
#classifying as spam or ham uisng a list of spam words
df2 = pd.DataFrame(columns=['text', 'spam'])
for i in tqdm(range(len(df))):
    spam = '0'
    for word in df.observation[i].split():
        text = df.observation[i]
        if word in ['now', 'urgent', 'do','ready', 'news', 'please', 'interested', 'call', 'right', 'sale', 'spent', 'vacation']:
            spam = '1'

    current_df2 = pd.DataFrame({'text': [text], 'spam': [spam]})
    df2 = df2.append(current_df2, ignore_index=True)


100%|██████████| 729/729 [00:00<00:00, 748.13it/s]


In [36]:
df2

,text,spam
0,7bit Bcc: shona.wilson@enron.com This meeting...,0
1,7bit Review available services with OATI for ...,0
2,"7bit Concerning traders news information, and...",1
3,"7bit As per our conversation, we will be meet...",0
4,7bit Traders News Meeting <true_name> Chuck <...,0
...,...,...
724,"7bit Bcc: jay.webb@enron.com, legal <.taylor@...",1
725,7bit Thought you may be interested in this as...,1
726,"7bit Bcc: debra.bailey@enron.com, rika.imai@e...",0
727,"7bit Bcc: paul.racicot@enron.com, sheila.twee...",0


In [37]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
import string
def process_text(text):
    # 1 Remove Punctuationa
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    # 2 Remove Stop Words
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

    # 3 Return a list of clean words
    return clean_words


df2['text'].head().apply(process_text)

0    [7bit, Bcc, shonawilsonenroncom, meeting, resc...
1    [7bit, Review, available, services, OATI, East...
2    [7bit, Concerning, traders, news, information,...
3    [7bit, per, conversation, meeting, truename, K...
4    [7bit, Traders, News, Meeting, truename, Chuck...
Name: text, dtype: object

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer=process_text).fit_transform(df2['text'])

In [42]:
#Split data into 80% training & 20% testing data sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(messages_bow, df2['spam'], test_size = 0.20, random_state = 0)

In [43]:
#Get the shape of messages_bow
messages_bow.shape

(729, 4514)

In [44]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [45]:
#Print the predictions
print(classifier.predict(X_train))
#Print the actual values
print(y_train.values)

['0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '1' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '0'
 '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '1' '0' '0' '0' '0' '0' '1' '0'
 '0' '0' '0' '0' '1' '1' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '1'
 '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0'
 '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0'
 '0' '1' '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '0

In [46]:
#Evaluate the model on the training data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(y_train ,pred ))
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy: ', accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       499
           1       0.96      0.90      0.93        84

    accuracy                           0.98       583
   macro avg       0.97      0.95      0.96       583
weighted avg       0.98      0.98      0.98       583

Confusion Matrix: 
 [[496   3]
 [  8  76]]

Accuracy:  0.9811320754716981


In [47]:
#Print the predictions
print('Predicted value: ',classifier.predict(X_test))
#Print Actual Label
print('Actual value: ',y_test.values)

Predicted value:  ['0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0'
 '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '1' '1' '0' '0' '0'
 '0' '0' '0' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '1' '0'
 '0' '0' '0' '0' '0' '1' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '1'
 '0' '0' '0' '1' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0']
Actual value:  ['0' '0' '0' '0' '1' '0' '1' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '1' '1' '0' '0' '0' '0'
 '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0'
 '0' '0'

In [48]:
#Evaluate the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93       122
           1       0.62      0.67      0.64        24

    accuracy                           0.88       146
   macro avg       0.77      0.79      0.78       146
weighted avg       0.88      0.88      0.88       146

Confusion Matrix: 
 [[112  10]
 [  8  16]]

Accuracy:  0.8767123287671232
